In [1]:
import numpy as np
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '5G'}, # I don't know what should be here
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4204276/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4204276/


23/07/30 16:13:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0067:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230730-1612-0.2.113-cf32652c5077.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from oligogenic_model import prep_pl_for_skat, run_skat_log, run_skat_for_mts

In [8]:
mts = []

for i in list(range(1,11)):
    
    mt = hl.read_matrix_table(
        localfs_path+'after_pca_no_filters_gts_vs_pol_subseted'+str(i)+'.mt'
    )
    mts.append(mt)

### Run SKAT for pl with zeros vs controls

#### annotate each of the mts with nearest genes and chekpoint:

In [ ]:
mts = prep_pl_for_skat(mts)

2023-07-30 16:20:20.827 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)
  Loading field 'f4' as type str (not specified)
  Loading field 'f5' as type str (not specified)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type str (not specified)
  Loading field 'f8' as type str (not specified)
2023-07-30 16:20:21.844 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)
2023-07-30 16:20:25.865 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-30 16:20:26.407 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-30 16:20:29.735 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-30 16:35:48.580 Hail

## Run SKAT for polish samples (9 different subsets) and GTS on scores, for all genes:

In [ ]:
mts = []

for i in list(range(10)):
    mt = hl.read_matrix_table(localfs_path+'_pl_'+str(i)+'annot.mt')
    mts.append(mt)

In [ ]:
len(mts)

In [ ]:
skat_tables, plots, options = run_skat_for_mts(mts)

In [ ]:
skat_tables = []
lambdas = []

for x in list(range(1,10)):
    for i in list(range(9)):
        
        if os.path.exists(
                localfs_path+'skat_table_'+str(x)+'_scores_no_filter_part_'+str(i)+'.mt'
            ):
            skat_table = hl.read_table(
                    localfs_path+'skat_table_'+str(x)+'_scores_no_filter_part_'+str(i)+'.mt'
                )
            print('skat_table_'+str(x)+'_scores_'+str(i)+'.mt')

            print(hl.lambda_gc(skat_table.p_value))
            lambdas.append(hl.lambda_gc(skat_table.p_value))
            skat_tables.append(skat_table)

            p = hl.plot.qq(skat_table.p_value)
            show(p)

In [ ]:
skat_tables = []

for x in list(range(20)):
    
    lambdas = []
    
    for i in list(range(9)):
        
        if os.path.exists(
                localfs_path+'skat_table_'+str(x)+'_scores_no_filter_part_'+str(i)+'.mt'
            ):
            skat_table = hl.read_table(
                    localfs_path+'skat_table_'+str(x)+'_scores_no_filter_part_'+str(i)+'.mt'
                )
            print('skat_table_'+str(x)+'_scores_no_filter_'+str(i)+'.mt')

            lambdas.append(hl.lambda_gc(skat_table.p_value))
            skat_tables.append(skat_table)
            
    print(lambdas)
    print(np.median(np.array(lambdas)))         

### Now run the same tests but in selected genes for best parameters

In [ ]:
mts = []

for i in list(range(9)):
    mt = hl.read_matrix_table(localfs_path+'_pl_'+str(i)+'annot.mt')
    mts.append(mt)

skat_tables = []
plots = []

gene_list = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2', 'NR2F1', 'ASH1L', 'FLT3']

In [ ]:
for idx, mt in enumerate(mts):
    mt = mt.filter_rows(hl.literal(gene_list).contains(mt.nearest_genes_20kb))
    skat_table, qq_plot = run_skat_log(mt, 8, 'scores_no_filter', '_polish_without_'+str(idx))
    skat_tables.append(skat_table)
    plots.append(qq_plot)

2023-07-15 23:15:20.505 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-15 23:15:22.328 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/3948668/tmp_hail/persist_tableMeMw5RFQbz
    Total size: 5.65 KiB
    * Rows: 5.64 KiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (55.00 B)
    * Largest partition:  1 rows (85.00 B)
2023-07-15 23:15:23.276 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/3948668/skat_table_8_scores_no_filter_polish_without_0.mt
2023-07-15 23:15:23.884 Hail: INFO: Coerced sorted dataset
2023-07-15 23:15:23.888 Hail: INFO: Coerced dataset with out-of-order partitions.


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHADL""",96,3.30e+03,2.84e-04,0
"""PCDH10""",236,4.03e+03,9.54e-04,0
"""DRD2""",216,3.50e+03,1.09e-03,0
"""SORCS3""",1318,1.87e+04,1.73e-03,0
"""CDH9""",448,4.43e+03,2.53e-03,0
"""MAOA""",6,8.71e+01,3.07e-03,0
"""PCDH1""",112,3.54e+03,3.69e-03,0
"""CD47""",169,2.37e+03,6.47e-03,0


2023-07-15 23:15:25.386 Hail: INFO: Coerced sorted dataset
2023-07-15 23:15:25.388 Hail: INFO: Coerced dataset with out-of-order partitions.
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-07-15 23:15:28.617 Hail: INFO: Coerced sorted dataset
2023-07-15 23:15:28.619 Hail: INFO: Coerced dataset with out-of-order partitions.


2023-07-15 23:26:27.886 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-15 23:26:30.817 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/3948668/tmp_hail/persist_tableJZ4xCKPUpt
    Total size: 5.65 KiB
    * Rows: 5.64 KiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (55.00 B)
    * Largest partition:  1 rows (85.00 B)
2023-07-15 23:26:31.438 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/3948668/skat_table_8_scores_no_filter_polish_without_1.mt
2023-07-15 23:26:32.132 Hail: INFO: Coerced sorted dataset
2023-07-15 23:26:32.133 Hail: INFO: Coerced dataset with out-of-order partitions.


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""PCDH10""",240,3.86e+03,3.36e-04,0
"""MAOA""",5,8.68e+01,6.28e-04,0
"""SORCS3""",1346,1.71e+04,1.05e-03,0
"""CHADL""",99,2.43e+03,2.30e-03,0
"""SLC6A4""",108,1.96e+03,4.45e-03,0
"""HTR2A""",245,3.07e+03,4.95e-03,0
"""SLITRK5""",137,1.15e+03,7.88e-03,0
"""CDH9""",460,3.35e+03,1.50e-02,0


2023-07-15 23:26:33.289 Hail: INFO: Coerced sorted dataset
2023-07-15 23:26:33.291 Hail: INFO: Coerced dataset with out-of-order partitions.
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-07-15 23:26:35.957 Hail: INFO: Coerced sorted dataset
2023-07-15 23:26:35.959 Hail: INFO: Coerced dataset with out-of-order partitions.


2023-07-15 23:36:28.025 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-07-15 23:36:29.401 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/3948668/tmp_hail/persist_table6yuKLjPPQL
    Total size: 5.65 KiB
    * Rows: 5.64 KiB
    * Globals: 11.00 B
    * Smallest partition: 1 rows (55.00 B)
    * Largest partition:  1 rows (85.00 B)
2023-07-15 23:36:30.022 Hail: INFO: wrote table with 75 rows in 75 partitions to /localfs/3948668/skat_table_8_scores_no_filter_polish_without_2.mt
2023-07-15 23:36:30.374 Hail: INFO: Coerced sorted dataset
2023-07-15 23:36:30.376 Hail: INFO: Coerced dataset with out-of-order partitions.


,,,,
id,size,q_stat,p_value,fault
str,int32,float64,float64,int32
"""CHADL""",97,4.31e+03,1.45e-05,0
"""DRD2""",218,4.56e+03,8.12e-05,0
"""PCDH10""",242,4.66e+03,5.33e-04,0
"""SORCS3""",1347,1.67e+04,4.80e-03,0
"""MAOA""",5,8.24e+01,5.89e-03,0
"""AADAC""",133,2.11e+03,7.58e-03,0
"""CDH9""",461,3.88e+03,1.17e-02,0
"""CHD8""",148,1.11e+03,1.70e-02,0


2023-07-15 23:36:31.442 Hail: INFO: Coerced sorted dataset
2023-07-15 23:36:31.444 Hail: INFO: Coerced dataset with out-of-order partitions.
/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/venv-hail-0.2.113/lib/python3.10/site-packages/bokeh/models/sources.py:235: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tmp_data = {c: v.values for c, v in _df.iteritems()}
2023-07-15 23:36:35.182 Hail: INFO: Coerced sorted dataset
2023-07-15 23:36:35.184 Hail: INFO: Coerced dataset with out-of-order partitions.


In [ ]:
skat_results = skat_tables[0]

skat_results = skat_results.select(
    skat_results.size
)

field_names = []

for i in list(range(9)):
    field_names.append('p_value_'+str(i))

skat_results = skat_results.annotate(
    **{
        f'p_value_{idx}': skat_tables[idx][skat_results.id].p_value
        for idx in list(range(9))
    }
)

skat_results.export('polish-without-zeros-8.tsv')